In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from google.colab import userdata


login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM





tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
input_text = "What is your name?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>What is your name?

What is your age?

What is your gender?

What


In [9]:
tokens = input_ids['input_ids']
print(tokens)

logits = model(**input_ids).logits
for i in range(tokens.shape[-1]):
    token = tokens[0, i].item()
    print(logits[0, i, token])

tensor([[     2,   1841,    603,    861,   1503, 235336]], device='cuda:0')
tensor(-18.2747, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-33.2665, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-23.9536, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-27.7627, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-19.6064, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-21.0372, device='cuda:0', grad_fn=<SelectBackward0>)


In [10]:
import torch

def zero_shot_classification(text, task_description, labels):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to("cuda")  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.
    probs = []
    for label in labels:  # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("cuda")
        n_label_tokens = label_ids['input_ids'].shape[-1] - 1  # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            'input_ids': torch.concatenate([text_ids['input_ids'], label_ids['input_ids'][:, 1:]], axis=-1),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            'attention_mask': torch.concatenate([text_ids['attention_mask'], label_ids['attention_mask'][:, 1:]], axis=-1)
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids['input_ids'].shape[-1]
        for i in range(n_label_tokens, 0, -1):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids['input_ids'][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

In [11]:
probs = zero_shot_classification("I am happy!", "Is the sentence positive or negative?: ", ["positive", "negative"])
print(probs)

[-4.5151824951171875, -9.59005069732666]


In [12]:
from datasets import load_dataset


imdb = load_dataset("imdb")
def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [15]:
import numpy as np
from tqdm import tqdm


n_corrects = 0
for i in tqdm(range(50)):
    text = tokenized_imdb['test'][i]['text']
    label = tokenized_imdb['test'][i]['label']
    probs = zero_shot_classification(
        text,
        "A movie review is given. Decide that the movie review is positive or negative: ",
        labels=["Answer: negative.", "Answer: positive."]
    )

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1

print(n_corrects)

100%|██████████| 50/50 [00:52<00:00,  1.06s/it]

44


##과제
이번 과제에서는 이전 주차 과제에서 활용했던 `fancyzhx/ag_news` 문제를 zero-shot classification으로 푸시면 됩니다. 아래 사항들에 유의하시면 될 것 같습니다.

- [ ]  Label들을 올바르게 text화 하여 넘겨주셔야 합니다.
- [ ]  `test` split data 50개에 대한 정확도 계산 코드 및 출력이 남아있어야 합니다.

이외에는 Gemma-2B 모델의 logit 계산 능력을 활용한다는 부분 빼고는 제약이 없습니다.


In [18]:
ag_news = load_dataset("fancyzhx/ag_news")

tokenized_ag_news = ag_news.map(preprocess_function, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [23]:
n_corrects = 0
for i in tqdm(range(20)):
    text = tokenized_ag_news['test'][i]['text']
    label = tokenized_ag_news['test'][i]['label']
    print("text -> ",text )
    print("label -> ",label )
    probs = zero_shot_classification(
        text,
        "A news is given. Decide that the news is positive or negative: ",
        labels=["Answer: negative.", "Answer: positive."]
    )

    pred = np.argmax(np.array(probs))
    print("pred ==> ", pred)
    if pred == label:
        n_corrects += 1

print(n_corrects)

print("n_corrects ==> " ,n_corrects)

  0%|          | 0/20 [00:00<?, ?it/s]

text ->  Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.
label ->  2


  5%|▌         | 1/20 [00:00<00:05,  3.43it/s]

pred ==>  0
text ->  The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.
label ->  3


 10%|█         | 2/20 [00:00<00:06,  2.60it/s]

...

pred ==>  0
text ->  Rocking the Cradle of Life When did life begin? One evidential clue stems from the fossil records in Western Australia, although whether these layered sediments are biological or chemical has spawned a spirited debate. Oxford researcher, Nicola McLoughlin, describes some of the issues in contention.
label ->  3


 95%|█████████▌| 19/20 [00:08<00:00,  2.45it/s]

pred ==>  0
text ->  Storage, servers bruise HP earnings update Earnings per share rise compared with a year ago, but company misses analysts' expectations by a long shot.
label ->  3


100%|██████████| 20/20 [00:08<00:00,  2.29it/s]

pred ==>  0
0
n_corrects ==>  0


In [22]:
n_corrects = 0
for i in tqdm(range(500)):
    text = tokenized_ag_news['test'][i]['text']
    label = tokenized_ag_news['test'][i]['label']
    probs = zero_shot_classification(
        text,
        "A news is given. Decide that the news is positive or negative: ",
        labels=["Answer: negative.", "Answer: positive."]
    )

    pred = np.argmax(np.array(probs))
    print("pred ==> ", pred)
    if pred == label:
        n_corrects += 1

print(n_corrects)

print("n_corrects ==> " ,n_corrects)

  0%|          | 1/500 [00:00<02:49,  2.95it/s]

pred ==>  0


  0%|          | 2/500 [00:00<03:29,  2.38it/s]

pred ==>  1


  1%|          | 3/500 [00:01<03:35,  2.31it/s]

...

pred ==>  0


100%|█████████▉| 499/500 [03:25<00:00,  2.42it/s]

pred ==>  0


100%|██████████| 500/500 [03:26<00:00,  2.42it/s]

pred ==>  0
151
n_corrects ==>  151


### 500개 뉴스 기사

결과 : 151 개

실제 pred 1 =>  60 개  (긍정)
실제 pred 0 =>  440 개  (나머지 부정)

어떤 의미인지???